### Imports


In [ ]:
import os
import glob
from collections import defaultdict
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import torch
from torch.nn import functional as F

In [ ]:
from neural_verification import (
    Tokenizer, 
    TransformerConfig, 
    Transformer, 
    cycle, 
    HeterogeneousDataLoader
)
from config import tokenizer_vocabulary, model_config

### Setup

In [ ]:
model = torch.load('model.pt')
tokenizer = Tokenizer(tokenizer_vocabulary)

In [ ]:
tok_embed = model.transformer.wte.weight.detach().cpu().numpy()
pos_embed = model.transformer.wpe.weight.detach().cpu().numpy()

In [ ]:
tokens = tokenizer_vocabulary
token_ids = []
for tok in tokenizer_vocabulary: 
    token_ids.append(tokenizer.encode([tok]))
print(token_ids)

In [ ]:
def get_closest_tokens_euclidean(embedding, embeddings, tokenizer, topk=10):
    # Calculate distances.
    distances = torch.norm(embeddings - embedding, dim=1)
    # Get the top k smallest distances.
    sims, indices = torch.topk(distances, topk, largest=False)
    
    # Decode the token IDs to tokens.
    closest_tokens = [tokenizer.decode([token_id]) for token_id in indices]
    return {
        sims[i].item(): closest_tokens[i]
        for i in range(len(closest_tokens))
    }

def get_closest_tokens_cosine(embedding, embeddings, tokenizer, topk=10):
    # Calculate similarities. We use cosine similarity here.
    similarities = F.cosine_similarity(embedding.unsqueeze(0), embeddings)
    # Get the top k indices.
    sims, indices = torch.topk(similarities, topk)
    
    # Decode the token IDs to tokens.
    closest_tokens = [tokenizer.decode([token_id]) for token_id in indices]
    return {
        sims[i].item(): closest_tokens[i]
        for i in range(len(closest_tokens))
    }

In [ ]:
embeddings = []

for token_id in token_ids:
    embeddings.append(tok_embed[token_id])
    
# Convert list to numpy arrays
embeddings = np.array(embeddings)